In [19]:
import pickle
import os
import sympy as sp

# load the scenario-properties-baseline.pkl file
with open('../scenario-properties-baseline.pkl', 'rb') as f:
    scenario_properties = pickle.load(f)

In [20]:
equations_flattened = [scenario_properties.full_coll[i, j] for j in range(scenario_properties.full_coll.cols) for i in range(scenario_properties.full_coll.rows)]
equations = [sp.lambdify(scenario_properties.all_symbolic_vars, eq, 'numpy') for eq in equations_flattened]

In [ ]:
x0 = scenario_properties.x0.T.values.flatten()

# pass the x0 values to the equations
values = [eq(*x0) for eq in equations]

# create a matrix, each column has the next 40 items of values
values 

values = [values[i:i+40] for i in range(0, len(values), 40)]

su_only = values[1]

collision_probability = 1 - np.exp(su_only)